In [ ]:
!pip install --upgrade xgboost openpyxl joblib


In [ ]:
# !pip install python-pptx==0.4.1
!pip install --upgrade python-pptx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: python-pptx
    Found existing installation: python-pptx 0.4.1
    Uninstalling python-pptx-0.4.1:
      Successfully uninstalled python-pptx-0.4.1


In [ ]:
# ===========================================
# 1. Configuración del Entorno
# ===========================================
import pandas as pd
import numpy as np

import os

from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from joblib import Parallel, delayed, dump, load
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# ===========================================
# 2. Carga y Exploración de Datos
# ===========================================
df = pd.read_excel('/content/Ejercicio Forecast ALL.xlsx')
df['fecha'] = pd.to_datetime(df['fecha'])
# df = df[df.codigoarticulo.isin(['AUACALM4',	'AUACBD1100',	'AUACBD850',	'AUACEG250',	'AUACFC350',	'AUACFX1000',
#                                 'AUACOWL504',	'AUACPB400',	'AUACRIM4F',	'AUACRP120',	'AUACSC901',	'AUACSH1000',
#                                 'AUELCR433',	'AUELMC3',	'AUELMC5',	'AUELME611',	'HEELAG1142KIT',	'HEELCA1012D',
#                                 'HEELCA2542D',	'HEELPW1770',	'HEELPW2275',	'HEELTB500',	'HEELVC0115P',	'HEELVC0640P',
#                                 'HEFUFC25',	'HEFUFCD12KIT',	'HEFUFCD21',	'HEFUFD52',	'HEFUFG71',	'MAEL2G100',	'MAEL2G40',
#                                 'MAEL2G65',	'SOELCSVM501',	'SOELCSVM510',	'SOELCSVM530',	'SOELPES6300',	'SOELSI7130MP',
#                                 'SOELSI7160XP',	'SOELSI7200XP',	'SOELSI8180MP',	'SOELSI8300MG',	'SOELSI9220DV',	'SOFUFW205CEL',
#                                 'SOFUFW33',	'SOSWC1-611',	'SOSWC1-811',	'SOSWM3-315',	'SOSWSWA2057',	'SOSWSWC690',	'SOSWSWW2060N'])]
df.drop_duplicates(inplace=True)
df = df.sort_values(['codigoarticulo', 'fecha']).reset_index(drop=True)
print(df.head())
print(df.info())
print(df.describe())

# ===========================================
# 3. Definición de Funciones
# ===========================================

# Función para identificar y manejar valores atípicos
def handle_outliers(data, column):
    """
    Identifica valores atípicos usando el método IQR y crea una nueva columna sin ellos.
    Luego, imputa los valores eliminados mediante interpolación.
    """
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 2* IQR
    upper_bound = Q3 + 2 * IQR

    # Crear una nueva columna con valores atípicos marcados como NaN
    #data[f'{column}1'] = data[column].apply(lambda x: x if lower_bound <= x <= upper_bound else np.nan)
    #-- Solo los atípicos superiores
    data[f'{column}1'] = data[column].apply(lambda x: x if  x <= upper_bound else np.nan)
    # Imputar valores NaN mediante interpolación
    data[f'{column}1'] = data[f'{column}1'].interpolate(method='linear')

    return data

# Función de creación de caracteristicas
def create_features(data, pandemia_start='2020-03-01', pandemia_end='2021-12-31', lags=[1,2,3]):
    df = data.copy()
    df['mes'] = df['fecha'].dt.month
    df['tendencia'] = np.arange(len(df))
    pandemia_start = pd.to_datetime(pandemia_start)
    pandemia_end = pd.to_datetime(pandemia_end)
    df['dummy_pandemia'] = ((df['fecha'] >= pandemia_start) & (df['fecha'] <= pandemia_end)).astype(int)

    #-- Variables Lag
    # for lag in lags:
    #     df[f'cantidad_lag_{lag}'] = df.groupby('codigoarticulo')['cantidad'].shift(lag)
    #df = df.dropna().reset_index(drop=True)

    #-- Variables potencia
    df['stock_cuadrado'] = df['stock'] ** 2
    df['margen_cuadrado'] = df['margen'] ** 2
    df['stock_cubo'] = df['stock'] ** 3
    df['margen_cubo'] = df['margen']**3
    df['stock_raiz'] = np.sqrt(df['stock'])
    df['margen_raiz'] = np.sqrt(df['margen'])

    #-- Variables logaritmicas
    df['stock_log'] = np.log(df['stock'] + 1)
    df['margen_log'] = np.log(df['margen'] + 1)

    df = pd.get_dummies(df, columns=['mes'], drop_first=True)
    return df

def check_and_create_missing_month_columns(df):
    """
    Verifica que el DataFrame tenga las columnas 'mes_1' a 'mes_12',
    y crea las que falten con valor False.

    Args:
        df (pd.DataFrame): El DataFrame a verificar.

    Returns:
        pd.DataFrame: El DataFrame con las columnas de mes completas.
    """
    mes_columns = [f'mes_{i}' for i in range(1, 13)]

    for mes_col in mes_columns:
        if mes_col not in df.columns:
            df[mes_col] = False  # Crea la columna con valor False
    return df


def guardar_mejor_modelo(modelo, codigoarticulo):
    """
    Guarda el modelo entrenado en un archivo .joblib en la carpeta modelos_guardados.
    """
    if not os.path.exists('modelos_guardados'):
        os.makedirs('modelos_guardados')

    path = f'modelos_guardados/modelo_{codigoarticulo}.joblib'
    dump(modelo, path)
    print(f'Modelo guardado: {path}')

def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred))
    diff = np.abs(y_true - y_pred)
    non_zero = denominator != 0
    return np.mean(2 * diff[non_zero] / denominator[non_zero]) * 100

def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    smape_val = smape(y_test, y_pred)
    return {'rmse': rmse, 'smape': smape_val, 'predictions': y_pred}

def process_articulo(articulo, data, features, target, preprocessor, tscv):
    df_art = data[data['codigoarticulo'] == articulo].copy().reset_index(drop=True)

    if len(df_art) < 18:
        return {'codigoarticulo': articulo, 'Mensaje': 'Datos insuficientes'}

    train_size = int(len(df_art) * 0.85)
    train_df, test_df = df_art.iloc[:train_size], df_art.iloc[train_size:]

    X_train_art, y_train_art = train_df[features], train_df[target]
    X_test_art, y_test_art = test_df[features], test_df[target]

    modelos = {
        'Regresión Lineal': Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', LinearRegression())
        ]),
        'Ridge Regression': GridSearchCV(Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', Ridge())
        ]), {'regressor__alpha': [0.1,0.5, 1.0, 5, 10.0,50, 100.0]}, cv=tscv, scoring='neg_root_mean_squared_error'),
        'Lasso Regression': GridSearchCV(Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', Lasso())
        ]), {'regressor__alpha': [0.001, 0.01, 0.1, 0.5,0.7, 1.0]}, cv=tscv, scoring='neg_root_mean_squared_error'),
        'ElasticNet Regression': GridSearchCV(Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', ElasticNet())
        ]), {'regressor__alpha': [0.01, 0.1,0.2,0.5,0.8, 1.0], 'regressor__l1_ratio': [0.1,0.2,0.35, 0.5,0.75, 0.8]}, cv=tscv, scoring='neg_root_mean_squared_error'),
        'Random Forest': Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
        ]),
        'XGBoost': Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42))
        ])
    }

    resultados_art = {'codigoarticulo': articulo}

    mejor_modelo = None
    mejor_rmse = float('inf')

    for nombre, modelo in modelos.items():
        try:
            if isinstance(modelo, GridSearchCV):
                modelo.fit(X_train_art, y_train_art)
                y_pred = modelo.predict(X_test_art)
            else:
                y_pred = evaluate_model(modelo, X_train_art, y_train_art, X_test_art, y_test_art)['predictions']

            rmse = np.sqrt(mean_squared_error(y_test_art, y_pred))
            smape_val = smape(y_test_art, y_pred)

            resultados_art[f'{nombre}_RMSE'] = rmse
            resultados_art[f'{nombre}_SMAPE'] = smape_val
            # Opcional: Almacenar las predicciones
            resultados_art[f'{nombre}_Predicciones'] = y_pred.tolist()


            if rmse < mejor_rmse:
                mejor_rmse = rmse
                mejor_modelo = modelo


        except Exception as e:
            resultados_art[f'{nombre}_Error'] = str(e)

    if mejor_modelo:
        guardar_mejor_modelo(mejor_modelo, articulo)

    return resultados_art

       fecha codigoarticulo  cantidad   stock  margen
0 2020-01-31       AUACALM4       181  1356.0   72.02
1 2020-02-29       AUACALM4       491  2123.0   67.15
2 2020-03-31       AUACALM4       467  1656.0   68.39
3 2020-04-30       AUACALM4        49  1607.0   75.81
4 2020-05-31       AUACALM4       160  1391.0   75.92
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3023 entries, 0 to 3022
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   fecha           3023 non-null   datetime64[ns]
 1   codigoarticulo  3023 non-null   object        
 2   cantidad        3023 non-null   int64         
 3   stock           3023 non-null   float64       
 4   margen          3022 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 118.2+ KB
None
                               fecha     cantidad         stock       margen
count                           3023  302

In [ ]:

# Ordenar los datos por 'codigoarticulo' y 'fecha'
df = df.sort_values(['codigoarticulo', 'fecha']).reset_index(drop=True)

# Aplicar la función de manejo de valores atípicos para cada codigoarticulo
df_cleaned = df.groupby('codigoarticulo').apply(lambda group: handle_outliers(group, 'cantidad'))


# Eliminar el índice adicional creado por groupby.apply
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned.sort_values(['codigoarticulo', 'fecha'], inplace=True)
df_cleaned['cantidad1'] = df_cleaned['cantidad1'].interpolate(method='backfill')
df_cleaned['margen'] = df_cleaned['margen'].interpolate(method='backfill')
df_cleaned.info()

# Crear las características
df_feat = create_features(df_cleaned)
print(df_feat.head())

# Definir las características y la variable objetivo
features = [
    'stock',
    'margen',
    #'tendencia',
    'dummy_pandemia',
    # 'cantidad_lag_1',
    # 'cantidad_lag_2',
    # 'cantidad_lag_3',
    'stock_cuadrado',
    'margen_cuadrado',
    'stock_cubo',
    'margen_cubo',
    'stock_raiz',
    'margen_raiz',
    'stock_log',
    'margen_log'
] + [col for col in df_feat.columns if 'mes_' in col]

target = 'cantidad1'

# Definir el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features)
    ]
)

# Definir TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=2)

# Obtener todos los códigos de artículo únicos
articulos = df_feat['codigoarticulo'].unique()

# Procesar en paralelo
resultados_finales = Parallel(n_jobs=-1)(
    delayed(process_articulo)(articulo, df_feat, features, target, preprocessor, tscv)
    for articulo in articulos
)

# Convertir los resultados a un DataFrame
df_resultados_finales = pd.DataFrame(resultados_finales)
print(df_resultados_finales.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3023 entries, 0 to 3022
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   fecha           3023 non-null   datetime64[ns]
 1   codigoarticulo  3023 non-null   object        
 2   cantidad        3023 non-null   int64         
 3   stock           3023 non-null   float64       
 4   margen          3023 non-null   float64       
 5   cantidad1       3023 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 141.8+ KB
       fecha codigoarticulo  cantidad   stock  margen  cantidad1  tendencia  \
0 2020-01-31       AUACALM4       181  1356.0   72.02      181.0          0   
1 2020-02-29       AUACALM4       491  2123.0   67.15      491.0          1   
2 2020-03-31       AUACALM4       467  1656.0   68.39      467.0          2   
3 2020-04-30       AUACALM4        49  1607.0   75.81       49.0          3   


In [ ]:
# prompt: Partiendo del Dataframe df_feat construye gráfico de línea que compare las variables 'cantidad' y 'cantidad1', haz esto por cada valor de 'codigoarticulo'

import matplotlib.pyplot as plt
import seaborn as sns

# Iterate through each unique 'codigoarticulo'
for codigo in df_feat['codigoarticulo'].unique():
    # Filter the DataFrame for the current 'codigoarticulo'
    df_temp = df_feat[df_feat['codigoarticulo'] == codigo]

    # Create the line plot
    plt.figure(figsize=(10, 6))  # Adjust figure size if needed
    sns.lineplot(x='fecha', y='cantidad', data=df_temp, label='cantidad')
    sns.lineplot(x='fecha', y='cantidad1', data=df_temp, label='cantidad1')

    # Customize the plot
    plt.title(f'Comparación de cantidad y cantidad1 para codigoarticulo {codigo}')
    plt.xlabel('Fecha')
    plt.ylabel('Cantidad')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.tight_layout()  # Adjust layout to prevent labels from overlapping
    plt.show()


In [ ]:
#-- Almacenamiento de las metricas de desempeño en los diferentes modelos
df_resultados_finales.to_excel('resultados_finales.xlsx', index=False)

In [ ]:
# Seleccionar un artículo para visualizar
#--- 'AUACSH1000' 'HEELAG1141' 'HEELPW1565' 'HEELXIW20'
articulo_ejemplo =  'HEFUFG71'  # Reemplaza con un código

def plot_modelos_articulos(articulo_ejemplo):
  # Filtrar los resultados para el artículo seleccionado
  resultado_articulo = df_resultados_finales[df_resultados_finales['codigoarticulo'] == articulo_ejemplo]

  # Verificar si hay predicciones disponibles
  modelos_disponibles = ['Regresión Lineal', 'Ridge Regression', 'Lasso Regression', 'ElasticNet Regression', 'Random Forest', 'XGBoost']
  predicciones = {}
  for modelo in modelos_disponibles:
      pred_col = f'{modelo}_Predicciones'
      if pred_col in resultado_articulo.columns:
          predicciones[modelo] = resultado_articulo.iloc[0][pred_col]

  # Verificar que haya al menos un modelo con predicciones
  if not predicciones:
      print(f"No hay predicciones disponibles para el artículo {articulo_ejemplo}.")
  else:
      # Obtener los datos reales
      df_art_ejemplo = df_feat[df_feat['codigoarticulo'] == articulo_ejemplo].copy().reset_index(drop=True)
      train_size_art = int(len(df_art_ejemplo) * 0.85)
      test_df_art = df_art_ejemplo.iloc[train_size_art:].copy()
      y_true = test_df_art[target].values

      # Crear DataFrame para visualizar las predicciones de diferentes modelos
      df_plot = pd.DataFrame({
          'fecha': test_df_art['fecha'],
          'Real': y_true
      })

      for modelo, preds in predicciones.items():
          df_plot[modelo] = preds

      # Graficar todas las predicciones
      plt.figure(figsize=(14,8))
      plt.plot(df_plot['fecha'], df_plot['Real'], label='Real', marker='o')

      for modelo in modelos_disponibles:
          if modelo in predicciones:
              plt.plot(df_plot['fecha'], df_plot[modelo], label=f'Predicción {modelo}', marker='x')

      plt.xlabel('Fecha')
      plt.ylabel('Cantidad')
      plt.title(f'Pronóstico de Cantidad para {articulo_ejemplo}')
      plt.legend()
      plt.xticks(rotation=45)
      plt.tight_layout()
      plt.show()

In [ ]:
# prompt: Almacena los gráficos generados en "# Obtener la lista de artículos únicos
# articulos = df_resultados_finales['codigoarticulo'].unique()
# # Iterar sobre la lista de artículos y llamar a la función para cada uno
# for articulo_ in articulos:
#     plot_modelos_articulos(articulo_)" dentro de un archivo de Power Point

from pptx import Presentation
from pptx.util import Inches

# Obtener la lista de artículos únicos
articulos = df_resultados_finales['codigoarticulo'].unique()
# Iterar sobre la lista de artículos y llamar a la función para cada uno
for articulo_ in articulos:
    plot_modelos_articulos(articulo_)

    # Crear una presentación de PowerPoint
    prs = Presentation()

    # Agregar una diapositiva con un título
    slide_layout = prs.slide_layouts[0]  # Usar el diseño de diapositiva de título
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    title.text = f"Gráfico de Predicciones para Artículo {articulo_}"

    # Agregar el gráfico generado a la diapositiva
    # Asumiendo que 'plt.gcf()' contiene el gráfico actual
    # Convertir el gráfico a una imagen
    plot_image = plt.gcf()
    plot_image.savefig('my_plot.png')

    # Agregar la imagen a la diapositiva
    left = Inches(1)
    top = Inches(2)
    width = Inches(8)
    height = Inches(6)
    pic = slide.shapes.add_picture('my_plot.png', left, top, width=width, height=height)

    # Guardar la presentación
    prs.save(f'predicciones_articulo_{articulo_}.pptx')

    # Limpiar el gráfico después de agregarlo a la presentación
    plt.clf()


In [ ]:
# Obtener la lista de artículos únicos
articulos = df_resultados_finales['codigoarticulo'].unique()

# Iterar sobre la lista de artículos y llamar a la función para cada uno
for articulo_ in articulos:
    plot_modelos_articulos(articulo_)



In [ ]:
def cargar_modelo_y_predecir(codigoarticulo, X_nuevo):
    """
    Carga el modelo guardado y predice para nuevos valores de X.
    """
    path = f'modelos_guardados/modelo_{codigoarticulo}.joblib'
    if not os.path.exists(path):
        raise FileNotFoundError(f'No se encontró un modelo guardado para el código de artículo {codigoarticulo}')

    modelo = load(path)
    return modelo.predict(X_nuevo)

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def pronosticar_variables_exog(articulo, data, k):
    """
    Pronostica k meses hacia adelante las variables stock y margen utilizando modelos de series de tiempo univariantes.
    """
    df_art = data[data['codigoarticulo'] == articulo].copy().reset_index(drop=True)
    resultados = {'codigoarticulo': articulo}

    for variable in ['stock', 'margen']:
        if df_art[variable].isna().sum() > 0:
            df_art[variable].fillna(method='ffill', inplace=True)

        # Group data by month and calculate the mean stock for each month
        monthly_stock = df_art.groupby(df_art['fecha'].dt.month)[variable].mean()

        if variable == 'stock':
            # Create a list to store the forecasted values
            predicciones = []
            for i in range(1, k + 1):
                # Get the month for the next k periods
                month = (df_art['fecha'].max().month + i) % 12
                if month == 0:
                    month = 12
                # Predict using the average stock of the corresponding month
                predicciones.append(monthly_stock.get(month, monthly_stock.mean()))

            predicciones = np.where(np.array(predicciones) < 0, 0, np.array(predicciones))
            predicciones = np.round(predicciones).astype(int)
            resultados[f'{variable}_pronostico'] = predicciones.tolist()

        else:
            # Check if enough data for ExponentialSmoothing
            if len(df_art) >= 24:  # Check if there are at least 24 months of data
                modelo = ExponentialSmoothing(df_art[variable], trend='add', seasonal='add', seasonal_periods=12).fit()
                predicciones = modelo.forecast(k)
            else:
                # If not enough data, use a simple average for forecasting
                predicciones = np.repeat(df_art[variable].mean(), k)

            predicciones = np.where(predicciones < 0, 0, predicciones)
            predicciones = np.round(predicciones).astype(int)
            resultados[f'{variable}_pronostico'] = predicciones.tolist()

    return resultados

In [ ]:
pronosticar_variables_exog(articulo = 'AUACSH1000', data = df_feat, k=6)

In [ ]:
# Obtener todos los códigos de artículo únicos
articulos = df_feat['codigoarticulo'].unique()

# Número de meses a pronosticar
k = 6

# Procesar en paralelo para cada artículo
resultados_pronostico = Parallel(n_jobs=-1)(
    delayed(pronosticar_variables_exog)(articulo, df_feat, k) for articulo in articulos
)

# Convertir los resultados a un DataFrame
df_pronostico = pd.DataFrame(resultados_pronostico)

# Crear un DataFrame para almacenar los pronósticos con fechas
pronosticos_con_fechas = []

for index, row in df_pronostico.iterrows():
  codigo_articulo = row['codigoarticulo']
  ultima_fecha = df_feat[df_feat['codigoarticulo'] == codigo_articulo]['fecha'].max()
  print(ultima_fecha)
  fechas_futuras = pd.date_range(start=ultima_fecha + pd.DateOffset(months=0), periods=k, freq='MS')
  print(fechas_futuras)
  for i in range(k):
    pronosticos_con_fechas.append({
        'codigoarticulo': codigo_articulo,
        'fecha': fechas_futuras[i],
        'stock_pronostico': row['stock_pronostico'][i],
        'margen_pronostico': row['margen_pronostico'][i]
    })

df_pronostico_fechas = pd.DataFrame(pronosticos_con_fechas)

# Mostrar el DataFrame con los pronósticos y las fechas
print(df_pronostico_fechas.head())


In [ ]:
df_pronostico_fechas.sample(5)

In [ ]:
#-- Creando las variables necesarias para los pronósticos
df_pronostico_fechas.rename(columns={'stock_pronostico': 'stock', 'margen_pronostico': 'margen'}, inplace=True)
df_features_tmp = create_features(df_pronostico_fechas)
df_features_pronostico = check_and_create_missing_month_columns(df_features_tmp)
df_features_pronostico

In [ ]:
# prompt: Toma el dataframe  df_features_pronostico como el archivo base para usar la función cargar_modelo_y_predecir para predecir por cada codigoarticulo

# Crear un DataFrame vacío para almacenar los pronósticos
pronosticos = []

# Iterar sobre cada código de artículo
for codigo in df_features_pronostico['codigoarticulo'].unique():
    # Filtrar el DataFrame para el código de artículo actual
    df_temp = df_features_pronostico[df_features_pronostico['codigoarticulo'] == codigo].copy()

    # Crear las características necesarias para la predicción
    X_nuevo = df_temp[features]

    # Cargar el modelo y predecir
    try:
        predicciones = cargar_modelo_y_predecir(codigo, X_nuevo)

        # Agregar los pronósticos al DataFrame
        for i in range(len(predicciones)):
            pronosticos.append({
                'codigoarticulo': codigo,
                'fecha': df_temp['fecha'].iloc[i],
                'cantidad_pronostico': predicciones[i]
            })
    except FileNotFoundError as e:
        print(e)  # Imprime el error si no se encuentra el modelo

# Convertir la lista de pronósticos en un DataFrame
df_pronosticos = pd.DataFrame(pronosticos)
df_pronosticos['cantidad_pronostico'] = np.where(df_pronosticos['cantidad_pronostico'] < 0, 0, df_pronosticos['cantidad_pronostico'])
df_pronosticos['cantidad_pronostico'] = df_pronosticos['cantidad_pronostico'].round().astype(int)
# Mostrar el DataFrame con los pronósticos
df_pronosticos.head(7)


In [ ]:
#-- Guardar datos pronósticados
df_pronosticos.to_excel('pronosticos_articulos.xlsx', index=False)

In [ ]:
# prompt: hacer una función para gráficar los valores de cantidad en el DataFrame df_feat y la variable cantidad_pronostico en el Dataframe  df_pronosticos para cada articulo

def plot_cantidad_vs_pronostico(articulo):
    """
    Grafica los valores de cantidad en df_feat y cantidad_pronostico en df_pronosticos para un artículo dado.
    """
    # Filtrar los DataFrames para el artículo específico
    df_feat_articulo = df_feat[df_feat['codigoarticulo'] == articulo]
    df_pronosticos_articulo = df_pronosticos[df_pronosticos['codigoarticulo'] == articulo]

    # Crear el gráfico
    plt.figure(figsize=(14, 6))
    plt.plot(df_feat_articulo['fecha'], df_feat_articulo['cantidad1'], label='Cantidad Real')
    plt.plot(df_pronosticos_articulo['fecha'], df_pronosticos_articulo['cantidad_pronostico'], label='Cantidad Pronosticada')
    plt.xlabel('Fecha')
    plt.ylabel('Cantidad')
    plt.title(f'Cantidad vs. Pronóstico para Artículo {articulo}')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
# prompt: Almacena los gráficos generados en "# Obtener la lista de artículos únicos
# articulos = df_pronosticos['codigoarticulo'].unique()
# # Iterar sobre la lista de artículos y llamar a la función para cada uno
# for articulo in articulos:
#     plot_cantidad_vs_pronostico(articulo)
# "  dentro de un archivo de Power Point

# ... (Your existing code) ...

# Obtener la lista de artículos únicos
articulos = df_pronosticos['codigoarticulo'].unique()
# Iterar sobre la lista de artículos y llamar a la función para cada uno
for articulo in articulos:
    plot_cantidad_vs_pronostico(articulo)

    # Crear una presentación de PowerPoint
    prs = Presentation()

    # Agregar una diapositiva con un título
    slide_layout = prs.slide_layouts[0]  # Usar el diseño de diapositiva de título
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    title.text = f"Gráfico de Cantidad vs Pronóstico para Artículo {articulo}"

    # Agregar el gráfico generado a la diapositiva
    # Convertir el gráfico a una imagen
    plot_image = plt.gcf()
    plot_image.savefig('my_plot.png')

    # Agregar la imagen a la diapositiva
    left = Inches(1)
    top = Inches(2)
    width = Inches(8)
    height = Inches(6)
    pic = slide.shapes.add_picture('my_plot.png', left, top, width=width, height=height)

    # Guardar la presentación
    prs.save(f'cantidad_vs_pronostico_articulo_{articulo}.pptx')

    # Limpiar el gráfico después de agregarlo a la presentación
    plt.clf()

# ... (Rest of your code) ...


In [ ]:
# Obtener la lista de artículos únicos
articulos = df_pronosticos['codigoarticulo'].unique()

# Iterar sobre la lista de artículos y llamar a la función para cada uno
for articulo in articulos:
    plot_cantidad_vs_pronostico(articulo)


In [ ]:
plot_cantidad_vs_pronostico(articulo = 'AUACSH1000')